### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# Files to Load 
readFileSchoolData = Path.cwd() / 'Resources' / 'schools_complete.csv'
readFileStudentData = Path.cwd() / 'Resources' / 'students_complete.csv'

# Read School and Student Data File and store into Pandas DataFrames
schooldf = pd.read_csv(readFileSchoolData)
studentdf = pd.read_csv(readFileStudentData)
completedf = pd.merge(studentdf, schooldf, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Create the "empty" dataframe for storing results. This will be a single place to store the output instead of
# keeping up with individual variables
outcolumns = ['Number of Schools', 
              'Number of Students', 
              'Budget', 
              'Average Math Score', 
              'Average Reading Score', 
              '% Passing Math', 
              '% Passing Reading', 
              '% Passing Both']
d = [[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]]
outputdf = pd.DataFrame(data=d)
outputdf.columns = outcolumns
outputdf = outputdf.set_axis(['District Summary'], axis='index')

# Calculate the total number of schools
schoolCount = len(completedf.school_name.unique())
outputdf['Number of Schools'][0] = schoolCount

# Calculate the total number of students
studentCount = completedf['Student ID'].count()
outputdf['Number of Students'][0] = studentCount

# Calculate the total budget of all schools in the district
outputdf['Budget'][0] = completedf.budget.unique().sum()

# Calculate the average math and reading scores
outputdf['Average Math Score'][0] = completedf.math_score.mean()
outputdf['Average Reading Score'][0] = completedf.reading_score.mean()

# Calculate the percent of students that are passing math and reading, separately
outputdf['% Passing Math'][0] = 100.00 * (completedf.loc[completedf["math_score"] >= 70.00,["math_score"]].count() / studentCount)
outputdf['% Passing Reading'][0] = 100.00 * (completedf.loc[completedf["reading_score"] >= 70.00,["reading_score"]].count() / studentCount)

# Calculate the percent of students that are passing BOTH math and reading
mathPass = completedf.loc[completedf["math_score"] >= 70, ['reading_score']]
outputdf['% Passing Both'][0] = 100 * (mathPass.loc[mathPass["reading_score"] >= 70, ['reading_score']].count() / studentCount)


In [3]:
# Perform the format mappings and output
outputdf['Number of Students'] = outputdf['Number of Students'].map("{:,}".format)
outputdf['Budget'] = outputdf['Budget'].map("${:,}".format)
outputdf['Average Math Score'] = outputdf['Average Math Score'].map("{:.2f}%".format)
outputdf['Average Reading Score'] = outputdf['Average Reading Score'].map("{:.2f}%".format)
outputdf['% Passing Math'] = outputdf['% Passing Math'].map("{:.2f}%".format)
outputdf['% Passing Reading'] = outputdf['% Passing Reading'].map("{:.2f}%".format)
outputdf['% Passing Both'] = outputdf['% Passing Both'].map("{:.2f}%".format)

outputdf

,Number of Schools,Number of Students,Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
District Summary,15.0,"39,170.0","$24,649,428.0",78.99%,81.88%,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [4]:
# Make the output dataframe and pull in the data from the schooldf dataframe
SchoolSummaryFeatures = ['School Type',
                        'Total Students',
                        'Total School Budget',
                        'Per Student Budget', 
                        'Average Math Score', 
                        'Average Reading Score', 
                        '% Passing Math', 
                        '%Passing Reading', 
                        '% Overall Passing']
SchoolSummaryIndex = schooldf['school_name'].values.tolist()
SchoolSummary_df = pd.DataFrame(columns=SchoolSummaryFeatures)
SchoolSummary_df = pd.DataFrame({"School Type": schooldf["type"], "Total Students":schooldf["size"], "Total School Budget":schooldf["budget"]})
SchoolSummary_df = SchoolSummary_df.set_axis(SchoolSummaryIndex, axis='index')
SchoolSummary_df.sort_index(axis=0, inplace=True)

# Perform the calculations that do not require looking at individual student performance
TotalStudents = SchoolSummary_df['Total Students']
SchoolSummary_df['Per Student Budget'] = SchoolSummary_df['Total School Budget'] / TotalStudents
AvgSchoolData = completedf.groupby(['school_name']).mean()
SchoolSummary_df['Average Math Score'] = AvgSchoolData['math_score']
SchoolSummary_df['Average Reading Score'] = AvgSchoolData['reading_score']
schoolNames = schooldf['school_name'].sort_values()
schoolNames = schoolNames.reset_index(drop=True)
SchoolPerPassMath = []
for school in schoolNames:
    SchoolPerPassMath.append(float((100 * completedf.loc[(completedf['school_name'] == school) & (completedf['math_score'] >= 70), ['school_name']].count() / TotalStudents[school])))

SchoolSummary_df['% Passing Math'] = SchoolPerPassMath
SchoolSummary_df.head(15)

SchoolPerPassRead = []
for school in schoolNames:
    SchoolPerPassRead.append(float((100 * completedf.loc[(completedf['school_name'] == school) & (completedf['reading_score'] >= 70), ['school_name']].count() / TotalStudents[school])))

SchoolSummary_df['% Passing Reading'] = SchoolPerPassRead
SchoolSummary_df.head(15)

SchoolPerPassBoth = []
for school in schoolNames:
    SchoolPerPassBoth.append(float((100 * completedf.loc[(completedf['school_name'] == school) & (completedf['reading_score'] >= 70) & (completedf['math_score'] >= 70), ['school_name']].count() / TotalStudents[school])))

SchoolSummary_df['% Overall Passing'] = SchoolPerPassBoth

output2df = SchoolSummary_df.copy()

In [5]:
# Perform the format mappings
output2df['Total Students'] = output2df['Total Students'].map("{:,}".format)
output2df['Total School Budget'] = output2df['Total School Budget'].map("{:,}".format)
output2df['Average Math Score'] = output2df['Average Math Score'].map("{:.2f}%".format)
output2df['Average Reading Score'] = output2df['Average Reading Score'].map("{:.2f}%".format)
output2df['% Passing Math'] = output2df['% Passing Math'].map("{:.2f}%".format)
output2df['% Passing Reading'] = output2df['% Passing Reading'].map("{:.2f}%".format)
output2df['% Overall Passing'] = output2df['% Overall Passing'].map("{:.2f}%".format)

output2df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","3,124,928",628.0,77.05%,81.03%,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","1,081,356",582.0,83.06%,83.98%,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","1,884,411",639.0,76.71%,81.16%,65.99%,80.74%,53.20%
Ford High School,District,"2,739","1,763,916",644.0,77.10%,80.75%,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","917,500",625.0,83.35%,83.82%,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","3,022,020",652.0,77.29%,80.93%,66.75%,80.86%,53.53%
Holden High School,Charter,427,"248,087",581.0,83.80%,83.81%,92.51%,96.25%,89.23%
Huang High School,District,"2,917","1,910,635",655.0,76.63%,81.18%,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","3,094,650",650.0,77.07%,80.97%,66.06%,81.22%,53.54%
Pena High School,Charter,962,"585,858",609.0,83.84%,84.04%,94.59%,95.95%,90.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [6]:
output3df = SchoolSummary_df.copy()

TopSchools = output3df.nlargest(5,'% Overall Passing')

In [7]:
TopSchools['Total Students'] = TopSchools['Total Students'].map("{:,}".format)
TopSchools['Total School Budget'] = TopSchools['Total School Budget'].map("{:,}".format)
TopSchools['Average Math Score'] = TopSchools['Average Math Score'].map("{:.2f}%".format)
TopSchools['Average Reading Score'] = TopSchools['Average Reading Score'].map("{:.2f}%".format)
TopSchools['% Passing Math'] = TopSchools['% Passing Math'].map("{:.2f}%".format)
TopSchools['% Passing Reading'] = TopSchools['% Passing Reading'].map("{:.2f}%".format)
TopSchools['% Overall Passing'] = TopSchools['% Overall Passing'].map("{:.2f}%".format)

TopSchools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","1,081,356",582.0,83.06%,83.98%,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","1,043,130",638.0,83.42%,83.85%,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","917,500",625.0,83.35%,83.82%,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","1,319,574",578.0,83.27%,83.99%,93.87%,96.54%,90.58%
Pena High School,Charter,962,"585,858",609.0,83.84%,84.04%,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [8]:
BotSchools = output3df.nsmallest(5,'% Overall Passing').sort_values(by = "% Overall Passing", ascending=False)

In [9]:
BotSchools['Total Students'] = BotSchools['Total Students'].map("{:,}".format)
BotSchools['Total School Budget'] = BotSchools['Total School Budget'].map("{:,}".format)
BotSchools['Average Math Score'] = BotSchools['Average Math Score'].map("{:.2f}%".format)
BotSchools['Average Reading Score'] = BotSchools['Average Reading Score'].map("{:.2f}%".format)
BotSchools['% Passing Math'] = BotSchools['% Passing Math'].map("{:.2f}%".format)
BotSchools['% Passing Reading'] = BotSchools['% Passing Reading'].map("{:.2f}%".format)
BotSchools['% Overall Passing'] = BotSchools['% Overall Passing'].map("{:.2f}%".format)

BotSchools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Johnson High School,District,"4,761","3,094,650",650.0,77.07%,80.97%,66.06%,81.22%,53.54%
Hernandez High School,District,"4,635","3,022,020",652.0,77.29%,80.93%,66.75%,80.86%,53.53%
Huang High School,District,"2,917","1,910,635",655.0,76.63%,81.18%,65.68%,81.32%,53.51%
Figueroa High School,District,"2,949","1,884,411",639.0,76.71%,81.16%,65.99%,80.74%,53.20%
Rodriguez High School,District,"3,999","2,547,363",637.0,76.84%,80.74%,66.37%,80.22%,52.99%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [10]:
# Create the dataframes for each of the grades

ninth_df = completedf.loc[(completedf['grade'] == '9th')]
ninthGroup = ninth_df.groupby(['school_name']).mean()
tenth_df = completedf.loc[(completedf['grade'] == '10th')]
tenthGroup = tenth_df.groupby(['school_name']).mean()
eleventh_df = completedf.loc[(completedf['grade'] == '11th')]
eleventhGroup = eleventh_df.groupby(['school_name']).mean()
twelveth_df = completedf.loc[(completedf['grade'] == '12th')]
twelvethGroup = eleventh_df.groupby(['school_name']).mean()

In [11]:
MathByGrade_df = pd.DataFrame(data={'9th':ninthGroup['math_score'], '10th':tenthGroup['math_score'], '11th':eleventhGroup['math_score'], '12th':twelvethGroup['math_score']})

In [12]:
MathByGrade_df['9th'] = MathByGrade_df['9th'].map("{:.2f}%".format)
MathByGrade_df['10th'] = MathByGrade_df['10th'].map("{:.2f}%".format)
MathByGrade_df['11th'] = MathByGrade_df['11th'].map("{:.2f}%".format)
MathByGrade_df['12th'] = MathByGrade_df['12th'].map("{:.2f}%".format)

MathByGrade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08%,77.00%,77.52%,77.52%
Cabrera High School,83.09%,83.15%,82.77%,82.77%
Figueroa High School,76.40%,76.54%,76.88%,76.88%
Ford High School,77.36%,77.67%,76.92%,76.92%
Griffin High School,82.04%,84.23%,83.84%,83.84%
Hernandez High School,77.44%,77.34%,77.14%,77.14%
Holden High School,83.79%,83.43%,85.00%,85.00%
Huang High School,77.03%,75.91%,76.45%,76.45%
Johnson High School,77.19%,76.69%,77.49%,77.49%


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [13]:
ReadByGrade_df = pd.DataFrame(data={'9th':ninthGroup['reading_score'], '10th':tenthGroup['reading_score'], '11th':eleventhGroup['reading_score'], '12th':twelvethGroup['reading_score']})

In [14]:
ReadByGrade_df['9th'] = ReadByGrade_df['9th'].map("{:.2f}%".format)
ReadByGrade_df['10th'] = ReadByGrade_df['10th'].map("{:.2f}%".format)
ReadByGrade_df['11th'] = ReadByGrade_df['11th'].map("{:.2f}%".format)
ReadByGrade_df['12th'] = ReadByGrade_df['12th'].map("{:.2f}%".format)

ReadByGrade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30%,80.91%,80.95%,80.95%
Cabrera High School,83.68%,84.25%,83.79%,83.79%
Figueroa High School,81.20%,81.41%,80.64%,80.64%
Ford High School,80.63%,81.26%,80.40%,80.40%
Griffin High School,83.37%,83.71%,84.29%,84.29%
Hernandez High School,80.87%,80.66%,81.40%,81.40%
Holden High School,83.68%,83.32%,83.82%,83.82%
Huang High School,81.29%,81.51%,81.42%,81.42%
Johnson High School,81.26%,80.77%,80.62%,80.62%


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [15]:
Spendbins = [0, 585, 630, 645, 660]
Spendbin_names = ['< $585', '$586 - 630', '$631 - 645', '> $646']

SchoolSummary_df['Spending Bins'] = pd.cut(SchoolSummary_df['Per Student Budget'], Spendbins, labels=Spendbin_names, include_lowest=True)
SpendingPerStudent_df = SchoolSummary_df.groupby(['Spending Bins']).mean()

In [16]:
SpendingPerStudent_df['Average Math Score'] = SpendingPerStudent_df['Average Math Score'].map("{:.2f}%".format)
SpendingPerStudent_df['Average Reading Score'] = SpendingPerStudent_df['Average Reading Score'].map("{:.2f}%".format)
SpendingPerStudent_df['% Passing Math'] = SpendingPerStudent_df['% Passing Math'].map("{:.2f}%".format)
SpendingPerStudent_df['% Passing Reading'] = SpendingPerStudent_df['% Passing Reading'].map("{:.2f}%".format)
SpendingPerStudent_df['% Overall Passing'] = SpendingPerStudent_df['% Overall Passing'].map("{:.2f}%".format)

SpendingPerStudent_df[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Bins,,,,,
< $585,83.46%,83.93%,93.46%,96.61%,90.37%
$586 - 630,81.90%,83.16%,87.13%,92.72%,81.42%
$631 - 645,78.52%,81.62%,73.48%,84.39%,62.86%
> $646,77.00%,81.03%,66.16%,81.13%,53.53%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [17]:
Sizebins = [0, 1650, 3000, 5000]
Sizebins_names = ['Small (< 1650)', 'Medium (1651 - 3000)', 'Large (> 3000)']

SchoolSummary_df['School Size Bins'] = pd.cut(SchoolSummary_df['Total Students'], Sizebins, labels=Sizebins_names, include_lowest=True)
StudentSize_df = SchoolSummary_df.groupby(['School Size Bins']).mean()

In [18]:
StudentSize_df['Average Math Score'] = StudentSize_df['Average Math Score'].map("{:.2f}%".format)
StudentSize_df['Average Reading Score'] = StudentSize_df['Average Reading Score'].map("{:.2f}%".format)
StudentSize_df['% Passing Math'] = StudentSize_df['% Passing Math'].map("{:.2f}%".format)
StudentSize_df['% Passing Reading'] = StudentSize_df['% Passing Reading'].map("{:.2f}%".format)
StudentSize_df['% Overall Passing'] = StudentSize_df['% Overall Passing'].map("{:.2f}%".format)

StudentSize_df[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size Bins,,,,,
Small (< 1650),83.60%,83.88%,93.44%,96.66%,90.33%
Medium (1651 - 3000),80.55%,82.68%,82.17%,89.63%,74.74%
Large (> 3000),77.06%,80.92%,66.46%,81.06%,53.67%


## Scores by School Type

* Perform the same operations as above, based on school type

In [19]:
SchoolType_df = SchoolSummary_df.groupby(['School Type']).mean()

In [20]:
SchoolType_df['Average Math Score'] = SchoolType_df['Average Math Score'].map("{:.2f}%".format)
SchoolType_df['Average Reading Score'] = SchoolType_df['Average Reading Score'].map("{:.2f}%".format)
SchoolType_df['% Passing Math'] = SchoolType_df['% Passing Math'].map("{:.2f}%".format)
SchoolType_df['% Passing Reading'] = SchoolType_df['% Passing Reading'].map("{:.2f}%".format)
SchoolType_df['% Overall Passing'] = SchoolType_df['% Overall Passing'].map("{:.2f}%".format)

SchoolType_df[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47%,83.90%,93.62%,96.59%,90.43%
District,76.96%,80.97%,66.55%,80.80%,53.67%
